In [2]:
from openpyxl import load_workbook, Workbook, cell
import pandas as pd, numpy as np
from FileProcessingFunctions import ifHeaderReturnIndex, isRowValid
from django.shortcuts import render
from django.contrib.auth.decorators import login_required
from django.http import HttpResponse
from prof.forms import FileSpaceForm
from prof.models import FileSpace, RunSpace, TransactionData
from django.http import HttpResponseRedirect
from django.core.urlresolvers import reverse
from guardian.shortcuts import get_objects_for_user, assign_perm, get_perms
from django.contrib.auth.models import User
from guardian.decorators import permission_required_or_403
from django.db import transaction



desiredColumns=('Order Number', 'Order Date', 'Business Unit', 'Customer Number', 'Customer Name', 'Product Number', 'Quantity',
                'List Price', 'Total Price', 'Discount', 'Invoice Amount')

lastColumnName = 'All Months'
transactionSheetName = 'TransactionData'

def processTransactionsFile(inputFile, sheetName, outputFile, desiredColumns):
    wb=load_workbook(inputFile, read_only=True, data_only=True)
    ws=wb.get_sheet_by_name(sheetName)
    output_wb=Workbook()
    output_ws=output_wb.active
    output_ws.title=sheetName
    revenueSwitch=True
    for row in ws.rows:
        if ifHeaderReturnIndex(row,desiredColumns):
            selectedIndex=ifHeaderReturnIndex(row, desiredColumns)
            outputColumnNames=[row[i].value for i in selectedIndex]
            output_ws.append(outputColumnNames)
            break
    for row in ws.rows:
        if ifHeaderReturnIndex(row,desiredColumns):
            pass
        elif isRowValid(row,selectedIndex):
            newrow=[row[i].value for i in selectedIndex]
            output_ws.append(newrow)
    output_wb.save(outputFile)
    return(outputFile)

def ProcessTransactionData(fileObject, run, sheetName=transactionSheetName,keyFileObject=None):
    if not(keyFileObject): keyFileObject = fileObject
    wb=load_workbook(fileObject.TheActualFile, read_only=True, data_only=True)
    ws=wb.get_sheet_by_name(sheetName)
    selectedIndex=None
    for row in ws.rows:
        if selectedIndex:
            if isRowValid(row,selectedIndex):
                newrow=[row[i].value for i in selectedIndex]
                dictionary=dict(zip(outputColumnNames,newrow))
                dictionary['UniqueRunID']=run
                dictionary['SourceFileObject']=keyFileObject
                TransactionData(**dictionary).save()
        else:
            if ifHeaderReturnIndex(row,desiredColumns):
                selectedIndex=ifHeaderReturnIndex(row, desiredColumns)
                outputColumnNames=[row[i].value.replace(" ", "") for i in selectedIndex]
    
    
def createKeyTemplate(transactionfile):
    Tr = pd.read_excel(transactionfile)
    Tr['Month']=Tr['Date'].apply(lambda x: str(x.year)+'-'+ ('0'+str(x.month))[-2:])
    KeyTemplate=Tr[['Business Unit','Product Number','Month', 'List Price']].groupby( ['Business Unit','Product Number','Month']).sum().round().unstack()
    KeyTemplate.columns=KeyTemplate.columns.droplevel(0)
    KeyTemplate.columns.name=None
    KeyTemplate[KeyTemplate!=np.nan]=None
    KeyTemplate=KeyTemplate.reset_index()
    KeyTemplate[lastColumnName] = np.nan
    return KeyTemplate
    
    
    
def createQuantityKey(transactionfile):
    Tr = pd.read_excel(transactionfile)
    Tr['Month']=Tr['Date'].apply(lambda x: str(x.year)+'-'+ ('0'+str(x.month))[-2:])
    QKey=Tr[['Business Unit','Product Number','Month', 'Quantity']].groupby( ['Business Unit','Product Number','Month']).sum().round().unstack()
    QKey.columns=QKey.columns.droplevel(0)
    QKey.columns.name=None
    #QKey[QKey!=np.nan]=None
    #QKey['Trailing Twelve Months'] = np.nan
    QKeyAgg=Tr[['Business Unit','Product Number', 'Quantity']].groupby( ['Business Unit','Product Number']).sum().round()
    QKeyAgg.rename(columns={'Quantity': lastColumnName},inplace=True)
    QKey=pd.concat([QKey,QKeyAgg],axis=1)
    QKey=QKey.reset_index().fillna(0)
    return QKey

In [ ]:
inputFile = 'SampleTransactionData.xlsx'
sheetName = 'TransactionData'
outputFile = 'TransactionD.xlsx'
processTransactionsFile(inputFile, sheetName, outputFile, desiredColumns)

In [ ]:
Tr = pd.read_excel(outputFile)
Tr.head()

In [ ]:
Key1=createKeyTemplate(outputFile)
Qkey=createQuantityKey(outputFile)
Key2=Tr[['Business Unit','Product Number', 'List Price']].groupby( ['Business Unit','Product Number']).sum().round()
Key2=Key2.reset_index()
Key1[lastColumnName]=Key2['List Price']
for column in Key1.iloc[:,2:-1].columns:
    Key1[column]=Key2['List Price']
Key1.head()

In [ ]:
KeyProd=(Key1.set_index(['Business Unit', 'Product Number'])*Qkey.set_index(['Business Unit', 'Product Number']))
KeyProd.head()

In [ ]:
normalizedpd=pd.concat([group/group.sum() for  _, group in KeyProd.groupby(level=0)])
normalizedpd#.to_pickle('NormalizedKey')

In [ ]:
PnLGroupedData=pd.read_pickle('PnLGroupedData')

In [26]:
trfile='TrDataSnippet.xlsx'
sheetName='TransactionData'
run=RunSpace.objects.get(pk=1)
TrFile=FileSpace.objects.get(FileName='SampleTransaction')


In [28]:
ProcessTransactionData(TrFile, run)

In [20]:
TrFile.save()
#TrFile.delete(keep_parents=True)

In [30]:
TrFile.TheActualFile

<FieldFile: ./SampleTransactionData.xlsx>

In [21]:

TransactionData(UniqueRunID=run)

<TransactionData: TransactionData object>

In [11]:
dictionary['UniqueRunID']=run
dictionary['SourceFileObject']=TrFile
dictionary

{'BusinessUnit': 'BU3',
 'CustomerName': 'Kbz Inc',
 'CustomerNumber': 'C-401',
 'Discount': 0,
 'InvoiceAmount': 8687.35,
 'ListPrice': 1241.05,
 'OrderDate': datetime.datetime(2016, 3, 18, 0, 0),
 'OrderNumber': 'O-4067',
 'ProductNumber': 'P-1007',
 'Quantity': 7,
 'SourceFileObject': <FileSpace: FileSpace object>,
 'TotalPrice': 8687.35,
 'UniqueRunID': <RunSpace: RunSpace object>}

In [9]:
[f.name for f in TransactionData._meta.get_fields()]

['id',
 'UniqueRunID',
 'SourceFileObject',
 'OrderNumber',
 'OrderDate',
 'BusinessUnit',
 'CustomerNumber',
 'CustomerName',
 'ProductNumber',
 'Quantity',
 'ListPrice',
 'TotalPrice',
 'Discount',
 'InvoiceAmount']

In [13]:
TransactionData(**dictionary).save()

In [14]:
selectedindex

NameError: name 'selectedindex' is not defined

In [24]:
for i in range(len(selectedIndex)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10


In [17]:
newrow

['O-4067',
 datetime.datetime(2016, 3, 18, 0, 0),
 'BU3',
 'C-401',
 'Kbz Inc',
 'P-1007',
 7,
 1241.05,
 8687.35,
 0,
 8687.35]

In [30]:
def method1():
    newrow=[row[i].value for i in selectedIndex]
    dictionary=dict(zip(outputColumnNames,newrow))
    dictionary['UniqueRunID']=run
    dictionary['SourceFileObject']=TrFile
    return TransactionData(**dictionary)    

def method2():
    TransactionData(UniqueRunID=run)
    TransactionData(SourceFileObject=TrFile)
    for i in range(len(selectedIndex)):
        TransactionData(outputColumnNames[i]=row[selectedIndex[i]].value)

SyntaxError: keyword can't be an expression (<ipython-input-30-7b3ead97a2a9>, line 12)

In [34]:
newrow=[row[i].value for i in selectedIndex]


In [35]:
newrow

[None, None, None, None, None, None, None, None, None, None, None]

In [3]:
from prof.models import FileSpace, RunSpace, TransactionData